<a href="https://colab.research.google.com/github/GuyRobot/AINotesBook/blob/main/EHealthVNCrawlerX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
step = 40
BASE_URL = "http://diendansuckhoe24h.net/"

# url = "https://suckhoetoday.com/p/179-Phong-benh-mua-dong.html/"
# url = "https://suckhoetoday.com/p/97-BENH-VA-CHUA-BENH-CHO-TRE-CON.html/"
# url = "https://suckhoetoday.com/p/98-BENH-DICH-THEO-MUA.html/"
# url = "https://suckhoetoday.com/p/185-Phong-benh-mua-he.html/"
url = "http://diendansuckhoe24h.net/"
# url = "https://suckhoetoday.com/p/93-CHE-DO-DINH-DUONG.html/"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
init_url = url

In [ ]:
soup.find_all("ol", {"id": "forums"})[0].find_all("ol", {"class": "nodeList"})[0].find_all("li", {"class": "node"})[0].find_all("h3", {"class": "nodeTitle"})[0].find_all("a")[0].get("href")

'forums/suc-khoe-tre-em/'

In [ ]:
ass = [a for x in soup.find_all("ol", {"id": "forums"})[0].find_all("ol", {"class": "nodeList"}) for a in x.find_all("li", {"class": "node"})]

In [27]:
pages = [a.find_all("h3", {"class": "nodeTitle"})[0].find_all("a")[0].get("href") for a in ass]

In [56]:
url = f"{BASE_URL}{pages[0]}"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [57]:
links = [a.find_all("h3", {"class": "title"})[0].find_all("a")[0].get("href") for a in soup.find_all("ol", {"class": "discussionListItems"})[0].find_all("li", {"class": "discussionListItem"})]

In [38]:
url = f"{BASE_URL}{links[0]}"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [43]:
ARTICLE = soup.find("div", {"class": "messageContent"}).get_text()

In [ ]:
import json
data = []
for p, page in enumerate(pages):
  print(f"Crawl {page}")
  try:
    page_url = f"{BASE_URL}{page}"
    page_page = urlopen(page_url)
    page_html = page_page.read().decode("utf-8")
    page_soup = BeautifulSoup(page_html, "html.parser")
    questions = [a.find_all("h3", {"class": "title"})[0].find_all("a")[0].get_text() for a in page_soup.find_all("ol", {"class": "discussionListItems"})[0].find_all("li", {"class": "discussionListItem"})]
    links = [a.find_all("h3", {"class": "title"})[0].find_all("a")[0].get("href") for a in page_soup.find_all("ol", {"class": "discussionListItems"})[0].find_all("li", {"class": "discussionListItem"})]
    for i, link in enumerate(links):
      url = f"{BASE_URL}{link}"
      page = urlopen(url)
      html = page.read().decode("utf-8")
      soup = BeautifulSoup(html, "html.parser")
      answer = soup.find("div", {"class": "messageContent"}).get_text()
      data.append({"question": re.sub(r'Question:', '', re.sub(r'(\n\s*)+\n+', '\n', re.sub(r'\r', r'\n', questions[i].strip())), re.I),
                        "answer": re.sub(r'Detailed Answer:', '', re.sub(r'Brief Answer:', '', re.sub(r'(\n\s*)+\n+', '\n', re.sub(r'\r', r'\n', answer.strip())), re.I), re.I), "url": url})
      if p % 2 == 0:
        with open("crawled.json", "w", encoding='utf-8') as f:
          json.dump(data, f, indent=4, ensure_ascii=False)
  except:
    pass

Crawl forums/suc-khoe-tre-em/


In [41]:
!pip install -q transformers

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [48]:
print(summarizer(ARTICLE[:512], max_length=512, min_length=30, do_sample=False))

Your max_length is set to 512, but your input_length is only 443. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=221)


[{'summary_text': 'Một ngày nào đó mình sẽ mua những món đồ chơi xịn cho bé như nhà bóng, xích đu, cỉu trượt về tại không gian sống nhè. Mỗi cái mà con yêu thích nhát.'}]


In [ ]:
links = [a.get("href") for a in soup.find_all("a", {"class": "title"})]
links